<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import torch
import torch.nn as nn
import torch.functional  as F

In [2]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels= in_channels, out_channels= out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
            nn.ReLU()
        )

    def forward(self, X):
        x = self.conv1(X)
        return x

In [3]:
class AlexNet(nn.Module):
    def __init__(self, num_classes):
        super(AlexNet, self).__init__()
        self.convs = nn.Sequential(
            ConvBlock(in_channels=3, out_channels=96, kernel_size=11, stride=4, padding=0),
            nn.MaxPool2d(96, kernel_size=3, stride=2, padding='valid'),

            ConvBlock(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2),
            nn.MaxPool2d(256, kernel_size=3, stride=2, padding='valid'),

            ConvBlock(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1),
            ConvBlock(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1),
            ConvBlock(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1),

            nn.MaxPool2d(256, kernel_size=3, stride=2, padding='valid')
        )
        self.linear = nn.Sequential(
            nn.Linear(in_features= 9216, out_features=4096),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(in_features=4096, out_features= 4096),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(in_features= 4096, out_features= num_classes),
            nn.LogSoftmax(dim=-1)
        )

    def forward(self, X):
        X = self.convs(X)
        X = X.view(X.shape[0], -1)
        X = self.linear(X)
        return X